<a href="https://colab.research.google.com/github/Cado87/Fruit-project/blob/main/Detection/YOLO/Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
#Connect to Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Unzip folder from /content/drive/MyDrive/Datasets/Fruits/Dataset_yolo.zip
!unzip -q /content/drive/MyDrive/Datasets/Fruits/Dataset_yolo.zip -d /content/

In [5]:
# Load a model
model = YOLO("yolo11n.pt")

# Validate the model
metrics = model.val(data="/content/Dataset_yolo/data.yaml")
print('mAP50-95:', metrics.box.map)  # map50-95

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs



FileNotFoundError: Dataset '/content/Dataset_yolo/data.yaml' images not found, missing path '/content/val.txt'
Note dataset download directory is '/content/datasets'. You can update this in '/root/.config/Ultralytics/settings.json'